<a href="https://colab.research.google.com/github/Rojanmhj2055/CIFAR10/blob/master/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext tensorboard
!rm -rf ./logs/

In [2]:
#The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, 
#with 6000 images per class. There are 50000 training images and 10000 test images.
import tensorflow as tf

from tensorflow import keras
import pandas
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from keras.datasets import cifar10
from keras.utils import np_utils

In [3]:
 (x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [4]:
import numpy as np

label = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]


In [5]:

y_train = np_utils.to_categorical(y_train,len(label))
y_test = np_utils.to_categorical(y_test,len(label))

In [6]:
x_train.shape

(50000, 32, 32, 3)

In [7]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [8]:
from keras.callbacks import LearningRateScheduler
n_labels = 10
def lr_schedule(epoch):
  lrate = 0.001
  if epoch>75:
    lr=0.0005
  elif epoch > 150:
    lrate = 0.0003
  return lrate   

In [9]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy']) 
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [10]:
def get_model():
  my_new_model =tf.keras.models.Sequential()

  my_new_model.add(keras.layers.Conv2D(64,(3,3),padding='same',activation='elu',input_shape=(32,32,3),kernel_regularizer=keras.regularizers.L2(0.0001)))
  my_new_model.add(keras.layers.Conv2D(64,(3,3),padding='same',activation='elu',input_shape=(32,32,3),kernel_regularizer=keras.regularizers.L2(0.0001)))
  my_new_model.add(keras.layers.BatchNormalization())
  my_new_model.add(keras.layers.MaxPool2D((2,2)))
  my_new_model.add(keras.layers.Dropout(0.3))

  my_new_model.add(keras.layers.Conv2D(128,(3,3),padding='same',activation="elu",kernel_regularizer=keras.regularizers.L2(0.0001)))
  my_new_model.add(keras.layers.Conv2D(128,(3,3),padding='same',activation="elu",kernel_regularizer=keras.regularizers.L2(0.0001)))
  my_new_model.add(keras.layers.BatchNormalization())
  my_new_model.add(keras.layers.MaxPool2D((2,2)))

  my_new_model.add(keras.layers.Conv2D(256,(3,3),padding='same',activation="elu",kernel_regularizer=keras.regularizers.L2(0.0001)))
  my_new_model.add(keras.layers.Conv2D(256,(3,3),padding='same',activation="elu",kernel_regularizer=keras.regularizers.L2(0.0001)))
  my_new_model.add(keras.layers.BatchNormalization())
  my_new_model.add(keras.layers.MaxPool2D(2,2))
  my_new_model.add(keras.layers.Dropout(rate=0.25))




  my_new_model.add(keras.layers.Conv2D(512,(3,3),padding='same',activation='elu',kernel_regularizer=keras.regularizers.l2(0.0001)))
  my_new_model.add(keras.layers.BatchNormalization())
  my_new_model.add(keras.layers.Conv2D(512,(3,3),padding='same',activation='elu',kernel_regularizer=keras.regularizers.l2(0.0001)))
  my_new_model.add(keras.layers.BatchNormalization())
  my_new_model.add(keras.layers.MaxPool2D((2,2)))
  my_new_model.add(keras.layers.Dropout(0.5))
  my_new_model.add(keras.layers.Flatten())

  my_new_model.add(keras.layers.Dense(512,activation="elu",kernel_regularizer= keras.regularizers.l2(0.0001)))
  my_new_model.add(keras.layers.BatchNormalization())
  my_new_model.add(keras.layers.Dropout(0.5))



  my_new_model.add(keras.layers.Dense(n_labels,activation="softmax"))
  
  return my_new_model

In [11]:
x_train.shape

(50000, 32, 32, 3)

In [12]:
model  = get_model()

optimizer = keras.optimizers.Adamax(lr=0.001,decay=1e-6)
schedular = keras.callbacks.LearningRateScheduler(lr_schedule)
# earlyStopping = keras.callbacks.EarlyStopping(monitor='accuracy',min_delta=0.5,patience=60,mode="max")
tensorboard_callback = tf.keras.callbacks.TensorBoard("/logs/mymodel",histogram_freq=1)
batch_size = 128
epoch = 200

In [13]:
#data agumentation 
import time
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip = True,
    rotation_range = 90,
  
)

datagen.fit(x_train)

model.compile(optimizer=optimizer,loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])
start = time.time()
history = model.fit(datagen.flow(x_train,y_train,batch_size=batch_size),steps_per_epoch=x_train.shape[0]//batch_size,epochs=epoch,
                           validation_data=(x_test,y_test), verbose=1,callbacks=[schedular,tensorboard_callback])
end=time.time()

Epoch 1/200
390/390 [==============================] - 39s 78ms/step - loss: 2.7626 - accuracy: 0.2610 - val_loss: 2.5870 - val_accuracy: 0.3262
Epoch 2/200
390/390 [==============================] - 29s 74ms/step - loss: 2.0307 - accuracy: 0.3961 - val_loss: 2.1013 - val_accuracy: 0.4368
Epoch 3/200
390/390 [==============================] - 29s 73ms/step - loss: 1.8126 - accuracy: 0.4561 - val_loss: 2.9418 - val_accuracy: 0.4646
Epoch 4/200
390/390 [==============================] - 28s 73ms/step - loss: 1.6370 - accuracy: 0.5199 - val_loss: 1.6348 - val_accuracy: 0.5495
Epoch 5/200
390/390 [==============================] - 28s 73ms/step - loss: 1.5282 - accuracy: 0.5565 - val_loss: 2.0787 - val_accuracy: 0.5269
Epoch 6/200
390/390 [==============================] - 28s 72ms/step - loss: 1.4289 - accuracy: 0.5963 - val_loss: 1.2672 - val_accuracy: 0.6512
Epoch 7/200
390/390 [==============================] - 28s 73ms/step - loss: 1.3305 - accuracy: 0.6268 - val_loss: 1.2278 - val_ac

In [ ]:
tensorboard --logdir="/logs/mymodel"

In [16]:
import pandas as pd
def to_csv(history,name):
  dicts={'loss':history.history['loss'],'accuracy':history.history['accuracy'],'validation loss':history.history['val_loss']
        , 'validation accuracy':history.history['val_accuracy']}

  df= pd.DataFrame(dicts)

  df.to_csv(name+".csv")
  return

to_csv(history,"my_model")

In [ ]:
!mkdir -p saved_model

In [17]:
model.save('drive/MyDrive/saved_model/my_model')

INFO:tensorflow:Assets written to: drive/MyDrive/saved_model/my_model/assets


In [18]:
print("The time to train the model:",end-start)

The time to train the model: 5704.889621734619


In [ ]:
import requests
from PIL import Image


In [ ]:
url ="https://ichef.bbci.co.uk/news/976/cpsprodpb/12A9B/production/_111434467_gettyimages-1143489763.jpg"
response = requests.get(url,stream=True)
print(response.status_code)
img = Image.open(response.raw)
img =tf.keras.preprocessing.image.img_to_array(img)
img = tf.image.resize(img,[32,32])
# img= tf.keras.preprocessing.image.array_to_img(img)

200


In [ ]:
def standardize(images,mean,std):
  images = (images - mean)/(std+1e-7)
  return image

In [ ]:
def show_image(images,labels,mean,std):
  
  fig = plt.figure(figsize=(25,10))

  for i in np.arange(20):
  
    ax = fig.add_subplot(2,10,i+1)
    plt.imshow(images[i])
    plt.subplots_adjust(bottom=0.2,hspace=0.3,wspace=0.1)
    my_image = tf.expand_dims(standardize(images[i],mean,std),axis=0)
    pred = label[np.argmax(model.predict(my_image))]
    l = label[np.argmax(labels[i])]
   
    ax.set_title('{} pred [{}]'.format(l,pred),color=("green" if pred==l else "red"))